# Decision process making : A quickie

In this notebook, we provide a quick walkthrough on the building process of a Sequential Decision Process (SDP) through this library.
Our study case will be a very dummy decision process with a single bounded state variable 's' and a single bounded action variable 'a'.
The dynamics is just s' = s + a and the cost function is p*-s' where p is a parameter.
Analytically, this means that a will be chosen to be its extremum value (which side depends on the parameter sign) at each time since it always yields the global optimal of the objective function.

Let us first build our variables:

In [40]:
from decision_process_components.variable import Variable
from decision_process_components.parameter import Parameter

s = Variable(support=(-1000, 1000))
a = Variable(support=(-1000, 1000))
p = Parameter()

# Notice how we can easily get a DTO from the variables and parameters
print(s.get_data().dict())
print(a.get_data().dict())
print(p.get_data().dict())

{'id': 'a5748e92-5261-4998-824f-665bda4f39f2', 'shape': (), 'v_type': <Type.REAL: (-inf, inf)>, 'support': (-1000.0, 1000.0)}
{'id': '6b40757a-c303-4851-9821-aee330e1b038', 'shape': (), 'v_type': <Type.REAL: (-inf, inf)>, 'support': (-1000.0, 1000.0)}
{'id': '31cab5e4-e67a-4fd6-83f0-54c9601baef0', 'shape': ()}


Notice how variables have unique ids when the latter is not specified. You can define them using id field in the constructor to facilitate model debugging.

Now we create our dynamics:

In [41]:
from decision_process_components.dynamics import Dynamics

# This dynamics might be seen as s_{t+1} = s_t + a_{t+1}
d = Dynamics(s, s + a)

Now let us imagine that we are not quite satisfied with the support of the action variable, which may be too permissive. We now build a constraint to squeeze a little bit that guy:

In [42]:
from decision_process_components.constraint import Constraint

maxp = Parameter()
# This constraint might be seen as a_t <= maxp_t
# (Note that a sequential parameter might be a repetitive constant)
c = Constraint(a <= maxp)

Now, could we constraint the parameter itself ? Let us check:

In [43]:
from decision_process_components.variable import NoVariableInvolvedError

maxp2 = Parameter()
try:
    c2 = Constraint(maxp <= maxp2)
except NoVariableInvolvedError as e:
    print("Blocked !")
    print(e)

Blocked !
No variable is involved in the current constraint. This is problematic in a decision process since the parameters are fixed in advance.


This has been blocked. This is expected since the value of the parameter comes from an external source (more on that later). A constraint should always involves variables.

Now let us go on and specify our cost function:

In [44]:
from decision_process_components.cost_function import CostFunction, LastTimeStepMask

r = CostFunction(p*s, horizon_mask=LastTimeStepMask())

So far we have gathered all the pieces of our puzzle. Let us now fully build our decision process model:

In [45]:
from decision_process import DecisionProcess

sdp = DecisionProcess("my_first_sdp")

sdp.add_state_variables(s)
sdp.add_action_variables(a)
# Helper variables (i.e., variables that are neither state and action but might come handy in a solution to perform some stuff with later on)
sdp.add_parameters(p, maxp)
sdp.add_dynamics_functions(d)
sdp.add_constraint_functions(c)
sdp.add_cost_functions(r)

print("Decision process built")

Decision process built


Finally, we validate our decision process model:

In [46]:
sdp.validate()

True

Yay! Our decision process pass all the tests. You can refer to the DecisionProcess docstring to see all the security checks, but let us exhibit one of them here. Suppose that we dare to use a non-defined variable in, say, a constraint:

In [47]:
from decision_process import NotDefinedVariableError

try:
    sdp.add_constraint_functions(Constraint(Variable() >= p))
except NotDefinedVariableError as e:
    print("blocked")
    print(e)

blocked
{'e629505b-09de-4337-975a-9d8fb29a5400'} variable(s) not defined in the decision process. Use add_state_variable, add_action_variable or even add_helper_variable if this is intended (according to the role(s) of the variable(s))


That's all folks. Well, that's all for decision processes. In a future notebook, we'll see how to actually obtain a realisation of this very dummy decision process, and to see that it outputs the solution that we have provided in an analytical form at the beginning of this notebook.